In [ ]:
!pip install openai pandas scikit-learn tensorflow
!pip install openai==0.28
!pip install gradio openai pandas scikit-learn keras joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0


In [ ]:

# === STEP 3: Embedding + Hybrid Prediction ===
def embed_texts(texts):
    response = openai.Embedding.create(model="text-embedding-ada-002", input=texts)
    return [item['embedding'] for item in response['data']]

def predict_duration(texts):
    embeddings = embed_texts(texts)
    dnn_outputs = dnn_model.predict(embeddings)
    combined_features = [embedding + list(dnn_out) for embedding, dnn_out in zip(embeddings, dnn_outputs)]
    duration_preds = rf_duration_model.predict(combined_features)
    return duration_preds

def predict_urgency(texts):
    embeddings = embed_texts(texts)
    urgency_preds = urgency_model.predict(embeddings)
    return urgency_preds


In [ ]:
!pip install ics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
import json

# （type）
task_types = [
    {
        "strategy_name": "Deep Work",
        "type": "type",
        "description": "Tasks that require long, uninterrupted concentration. Best scheduled in the morning.",
        "task_traits": ["analysis", "synthesis", "long reading", "coding", "academic writing"],
        "recommended_time": "Morning (8am–12pm)"
    },
    {
        "strategy_name": "Medium Focus",
        "type": "type",
        "description": "Tasks that need attention but tolerate some interruption. Suitable for early afternoon.",
        "task_traits": ["review", "light research", "note-taking", "editing"],
        "recommended_time": "Early Afternoon (1pm–3pm)"
    },
    {
        "strategy_name": "Creative Work",
        "type": "type",
        "description": "Tasks that require idea generation or creative expression. Often best in late morning or night.",
        "task_traits": ["brainstorm", "design", "writing lyrics", "project ideation"],
        "recommended_time": "Late Morning or Night"
    },
    {
        "strategy_name": "Low Focus",
        "type": "type",
        "description": "Repetitive or simple tasks. Can be scheduled during low energy slots.",
        "task_traits": ["email", "scheduling", "data entry", "cleaning files"],
        "recommended_time": "Late Afternoon (3pm–6pm) or Evening"
    }
]

#（method）
task_methods = [
    {
        "strategy_name": "Pomodoro Technique",
        "type": "method",
        "suitable_types": ["Medium Focus", "Low Focus"],
        "description": "25 minutes work + 5 minutes break cycle to maintain momentum.",
        "application_steps": [
            "Work for 25 minutes without distraction.",
            "Take a 5-minute break.",
            "After 4 cycles, take a 15–30 minute break."
        ]
    },
    {
        "strategy_name": "Time Blocking",
        "type": "method",
        "suitable_types": ["Deep Work", "Creative Work"],
        "description": "Divide day into blocks for specific types of tasks.",
        "application_steps": [
            "Create visual layout of your day.",
            "Reserve blocks for deep or creative tasks.",
            "Protect time from interruption."
        ]
    },
    {
        "strategy_name": "Energy Curve Matching",
        "type": "method",
        "suitable_types": ["Deep Work", "Medium Focus"],
        "description": "Align task difficulty with energy peaks.",
        "application_steps": [
            "Identify personal peak energy times.",
            "Schedule complex tasks during peak.",
            "Do simple tasks during energy dips."
        ]
    },
    {
        "strategy_name": "Habit Stacking",
        "type": "method",
        "suitable_types": ["Low Focus"],
        "description": "Attach task to an existing habit.",
        "application_steps": [
            "Pick a habit you already do.",
            "Define a micro-task to follow it.",
            "Repeat daily until automatic."
        ]
    }
]

full_strategy_knowledge = task_types + task_methods

with open("task_strategy_knowledge_base.json", "w") as f:
    json.dump(full_strategy_knowledge, f, indent=4)

print("saved task_strategy_knowledge_base.json")

saved task_strategy_knowledge_base.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ics import Calendar, Event
import datetime

def export_to_ics(week_df):
    cal = Calendar()
    base_date = datetime.date.today()
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    for i, row in week_df.iterrows():
        time_str = row["Time"]
        for day in weekdays:
            content = row[day]
            if content.strip():
                try:
                    title, duration_str = content.split("｜")
                    duration_hours = float(duration_str.replace("h", "").strip())
                except:
                    title = content.strip()
                    duration_hours = 1

                start_time = datetime.datetime.strptime(f"{base_date + datetime.timedelta(days=weekdays.index(day))} {time_str}", "%Y-%m-%d %H:%M")
                end_time = start_time + datetime.timedelta(hours=duration_hours)

                e = Event()
                e.name = title
                e.begin = start_time
                e.end = end_time
                cal.events.add(e)

    with open("/mnt/data/schedule.ics", "w") as f:
        f.writelines(cal)
    return "/mnt/data/schedule.ics"

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import json, os, datetime
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import load_model
from keras.losses import MeanSquaredError
import joblib
from openai.embeddings_utils import get_embedding
import openai
from typing import List, Tuple

# === Load OpenAI key ===
os.environ["OPENAI_API_KEY"] ="" # Enter your key
openai.api_key = os.environ["OPENAI_API_KEY"]

# === Load models ===

dnn_model = load_model("your_dnn_model.h5", custom_objects={'mse': MeanSquaredError()}) # Enter your path
rf_duration_model = joblib.load("your_rf_duration_model.pkl") # Enter your path
urgency_model = joblib.load("your_rf_urgency_model.pkl") # Enter your path


# === Knowledge Base Loader ===

def load_knowledge_base(kb_path="task_strategy_knowledge_base.json"):
    with open(kb_path, 'r') as f:
        full_kb = json.load(f)
    task_types = {i["strategy_name"]: i for i in full_kb if i["type"] == "type"}
    task_methods = {i["strategy_name"]: i for i in full_kb if i["type"] == "method"}
    return task_types, task_methods

def embed_texts(texts):
    response = openai.Embedding.create(model="text-embedding-ada-002", input=texts)
    return [item['embedding'] for item in response['data']]

def prepare_type_embeddings(task_types):
    keys = list(task_types.keys())
    texts = [f"{k}: {task_types[k]['description']}" for k in keys]
    return keys, embed_texts(texts)

def retrieve_task_type(query, type_keys, type_embeddings):
    q_emb = embed_texts([query])[0]
    sims = cosine_similarity([q_emb], type_embeddings)[0]
    best = np.argmax(sims)
    return type_keys[best]

def calculate_priority_score(est, urg, days_left):
    norm_est = min(est / 10, 1)
    norm_days = min(days_left / 60, 1)
    score = 0.4 * norm_est + 0.4 * urg + 0.2 * (1 - norm_days)
    return round(min(max(score, 0), 1), 2)

def recommend_strategy(task, types, methods, type_keys, type_embeddings):
    t_type = retrieve_task_type(task, type_keys, type_embeddings)
    method = next((m for m, v in methods.items() if t_type in v.get("suitable_types", [])), None)
    return t_type, method or "None"

def run_batch_prediction(task_df, kb_path="task_strategy_knowledge_base.json"):
    # Load knowledge base and semantic type embeddings
    types, methods = load_knowledge_base(kb_path)
    type_keys, type_embeddings = prepare_type_embeddings(types)

    # Prepare full texts for batch embedding
    texts = [f"{str(row.get('Title', ''))} {str(row.get('Description', ''))}" for _, row in task_df.iterrows()]
    embeddings = embed_texts(texts)

    # Duration prediction via DNN + Random Forest
    dnn_outputs = dnn_model.predict(embeddings)
    combined_features = [embedding + list(dnn_out) for embedding, dnn_out in zip(embeddings, dnn_outputs)]
    duration_preds = rf_duration_model.predict(combined_features)

    # Urgency prediction via RF model
    urgency_preds = urgency_model.predict(embeddings)

    results = []
    for i, (_, row) in enumerate(task_df.iterrows()):
        title = str(row.get("Title", ""))
        desc = str(row.get("Description", ""))
        date_str = row.get("Deadline", "")
        try:
            deadline = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
        except:
            continue
        days_left = max((deadline - datetime.date.today()).days, 1)

        # Retrieve predicted values
        est_hours = float(duration_preds[i])
        urg_level = float(urgency_preds[i])
        priority = calculate_priority_score(est_hours, urg_level, days_left)

        # Recommend task type and method from knowledge base
        task_type, method = recommend_strategy(texts[i], types, methods, type_keys, type_embeddings)

        results.append({
            "Title": title,
            "Deadline": date_str,
            "Estimated Hours": round(est_hours, 1),
            "Urgency": round(urg_level, 2),
            "Task Type": task_type,
            "Method": method,
            "Priority": priority
        })

    df = pd.DataFrame(results)
    selector_choices = [f"{row['Title']}｜{row['Estimated Hours']}h" for _, row in df.iterrows()]
    return df, selector_choices
# === Weekly View ===
def generate_week_table(date_str):
    try:
        selected = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return "Invalid date."
    start = selected - datetime.timedelta(days=selected.weekday())
    days = [(start + datetime.timedelta(days=i)).strftime("%A (%Y-%m-%d)") for i in range(7)]
    hours = [f"{h:02}:00" for h in range(24)]
    html = "<table border='1'><tr><th>Time</th>" + ''.join(f"<th>{d}</th>" for d in days) + "</tr>"
    for h in hours:
        html += f"<tr><td>{h}</td>" + "".join("<td></td>" for _ in days) + "</tr>"
    html += "</table>"
    return html

# === UI ===
with gr.Blocks() as demo:
    gr.Markdown("# Task Strategy Planner with RAG")

    with gr.Tab(" Batch Upload & Predict"):
        task_input = gr.Dataframe(headers=["Title", "Description", "Deadline"], row_count=5)
        predict_btn = gr.Button("Run RAG Model")
        prediction_output = gr.Dataframe()


# --- Editable DataFrame init func ---
    def init_week_table():
      hours = [f"{h:02}:00" for h in range(24)]
      days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
      columns = ["Time"] + days
      data = [[hour] + ["" for _ in days] for hour in hours]
      return pd.DataFrame(data, columns=columns)

    with gr.Tab(" Weekly Planner"):
        week_table = gr.Dataframe(
          value=init_week_table(),
          headers=["Time", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
          row_count=24,
          col_count=(8, "fixed"),
          label="Editable Weekly Schedule"
    )

        gr.Markdown("enter（form：task｜description｜duration）")

        save_btn = gr.Button(" Save Schedule")
        save_output = gr.Textbox(label="Save Status")
        gr.Markdown("### Add AI output into schedule")

        task_selector = gr.Dropdown(label="Choose task", choices=[])
        edited_task = gr.Textbox(label="task")
        edited_duration = gr.Number(label="duration", value=1.0)

        day_selector = gr.Dropdown(label="choose week", choices=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
        start_time_selector = gr.Dropdown(label="start time", choices=[f"{h:02}:00" for h in range(24)])

        insert_btn = gr.Button("insert table")
        insert_status = gr.Textbox(label="status")
        export_ics_btn = gr.Button(" export Google Calendar（.ics）")
        ics_path = gr.File(label="download .ics ")

        export_ics_btn.click(fn=export_to_ics, inputs=[week_table], outputs=[ics_path])
    save_btn.click(fn=lambda x: " Weekly schedule saved." if x else " No data.", inputs=[week_table], outputs=[save_output])

    predict_btn.click(fn=run_batch_prediction, inputs=[task_input], outputs=[prediction_output])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7b4b8ffbbea86e9ba2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
